In [149]:
import pandas as pd
import numpy as np

#import packages for data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Define the color palette
palette = {0: 'indianred', 1: 'steelblue'}

#load the dataset "final_dataset_Nov_9.csv"
df = pd.read_csv("../Lee/Cleaning and Models - First Attempt/lee_full_dataset.csv")

df.drop(columns=['MAJOR'],inplace=True)
df.rename(columns={'GRADUATE':'Y'},inplace=True)
df = df.astype(int)
df

,STUDENT,104,105,140,143,145,150,151,160,165,...,317,350,365,373,385,414,415,435,436,Y
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,5,0,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9369,13059,0,0,3,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9370,13060,0,0,-1,0,0,-3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9371,13061,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9372,13062,0,0,0,0,0,-4,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [150]:
courses = df.columns.to_list()[1:-1]
print(len(courses))
courses

28


['104',
 '105',
 '140',
 '143',
 '145',
 '150',
 '151',
 '160',
 '165',
 '166',
 '201',
 '207',
 '265',
 '266',
 '267',
 '301',
 '302',
 '304',
 '314',
 '317',
 '350',
 '365',
 '373',
 '385',
 '414',
 '415',
 '435',
 '436']

In [151]:
grouped = df.groupby(courses)
df_grouped = pd.DataFrame({
    'Pr(X)' : grouped.size() / len(df),
    'Pr(Y|X)': grouped['Y'].mean()}).reset_index()
df_grouped

,104,105,140,143,145,150,151,160,165,166,...,350,365,373,385,414,415,435,436,Pr(X),Pr(Y|X)
0,-8,0,-5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000107,1.0
1,-8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000320,1.0
2,-8,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000107,0.0
3,-8,0,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000107,0.0
4,-7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.001067,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2361,8,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0.000107,0.0
2362,8,0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000107,1.0
2363,8,0,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000213,1.0
2364,8,0,7,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.000213,1.0


In [152]:
df_grouped['Pr(Y|X)'].value_counts().sort_index()

Pr(Y|X)
0.000000     916
0.078740       1
0.100000       1
0.111111       1
0.125000       3
            ... 
0.866667       1
0.875000       2
0.888889       1
0.900000       1
1.000000    1111
Name: count, Length: 110, dtype: int64

In [153]:
#compare the dot product of Pr(Y|X) and Pr(X) with the mean value of Y in df
np.dot(df_grouped['Pr(Y|X)'], df_grouped['Pr(X)']), df['Y'].mean()

(np.float64(0.5138681459355668), np.float64(0.5138681459355665))

In [154]:
#add a column named "ERROR(Y|X)" which contains the minimum of Pr(Y|X) and 1-Pr(Y|X)
df_grouped['ERROR(Y|X)'] = np.minimum(df_grouped['Pr(Y|X)'], 1-df_grouped['Pr(Y|X)'])

#compute the bayes error rate. This is the expected value of ERROR(Y|X) over the distribution of X
bayes_error_rate = np.dot(df_grouped['ERROR(Y|X)'], df_grouped['Pr(X)'])

print(f'Maximum achievable accuracy: {(1-bayes_error_rate)*100:.3f}%')

Maximum achievable accuracy: 73.437%


In [155]:
df_grouped.sum()

104            358.000000
105            105.000000
140            607.000000
143            562.000000
145             76.000000
150            344.000000
151            124.000000
160            190.000000
165           1319.000000
166           1696.000000
201           1978.000000
207           1363.000000
265           1989.000000
266            964.000000
267           1510.000000
301           1625.000000
302            358.000000
304            589.000000
314            770.000000
317           2094.000000
350            540.000000
365            488.000000
373            582.000000
385            864.000000
414           1706.000000
415            430.000000
435            890.000000
436            966.000000
Pr(X)            1.000000
Pr(Y|X)       1278.793426
ERROR(Y|X)     123.503466
dtype: float64

In [156]:
#import machine learning libraries
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

In [157]:
X = df[courses]
y = df['Y']

In [158]:
X.columns

Index(['104', '105', '140', '143', '145', '150', '151', '160', '165', '166',
       '201', '207', '265', '266', '267', '301', '302', '304', '314', '317',
       '350', '365', '373', '385', '414', '415', '435', '436'],
      dtype='object')

In [159]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2, 
                                                    random_state=42,
                                                    shuffle=True,
                                                    stratify=y)

In [160]:
from sklearn.feature_selection import SelectKBest, f_classif
"""
This script performs feature selection and model evaluation using various classifiers with a pipeline approach.

The script includes the following steps:
1. Import necessary libraries and modules.
2. Define the number of features to select using SelectKBest.
3. Create a dictionary of classifiers, each with a pipeline that includes feature selection and the classifier.
4. Perform 5-fold cross-validation for each model.
5. Print the cross-validation results, including mean accuracy and standard deviation for each model.

Classifiers included:
- Linear Discriminant Analysis (LDA)
- Logistic Regression
- Linear Support Vector Classifier (SVC)
- Quadratic Discriminant Analysis (QDA)
- Polynomial LDA
- Polynomial Logistic Regression
- Gaussian Naive Bayes (GNB)
- K-Nearest Neighbors (KNN)
- Radial Basis Function (RBF) SVC
- Random Forest

Parameters:
- k (int): The number of features to select using SelectKBest.

Possible values for the parameter "scoring" in cross_val_score:
- 'accuracy'
- 'adjusted_rand_score'
- 'average_precision'
- 'balanced_accuracy'
- 'brier_score_loss'
- 'f1'
- 'f1_micro'
- 'f1_macro'
- 'f1_weighted'
- 'f1_samples'
- 'neg_log_loss'
- 'precision'
- 'recall'
- 'roc_auc'
- 'roc_auc_ovr'
- 'roc_auc_ovo'
- 'roc_auc_ovr_weighted'
- 'roc_auc_ovo_weighted'
- 'neg_mean_absolute_error'
- 'neg_mean_squared_error'
- 'neg_root_mean_squared_error'
- 'r2'
- 'explained_variance'
- 'max_error'
- 'neg_median_absolute_error'
- 'neg_mean_poisson_deviance'
- 'neg_mean_gamma_deviance'
"""
from sklearn.pipeline import Pipeline

# Define the number of features to select
k = 10


# Update classifiers to include feature selection
classifiers = {
    'lda': Pipeline([('select', SelectKBest(f_classif, k=k)), ('lda', LinearDiscriminantAnalysis())]),
    'log_reg': Pipeline([('select', SelectKBest(f_classif, k=k)), ('log_reg', LogisticRegression(max_iter=100000, random_state=42))]),
    'svc_linear': Pipeline([('select', SelectKBest(f_classif, k=k)), ('svc_linear', LinearSVC(dual='auto'))]),
    'qda': Pipeline([('select', SelectKBest(f_classif, k=k)), ('qda', QuadraticDiscriminantAnalysis())]),
    'lda_poly': Pipeline([('scale', StandardScaler()), ('poly', PolynomialFeatures(2)), ('select', SelectKBest(f_classif, k=k)), ('lda', LinearDiscriminantAnalysis())]),
    'log_reg_poly': Pipeline([('scale', StandardScaler()), ('poly', PolynomialFeatures(2)), ('select', SelectKBest(f_classif, k=k)), ('log_reg', LogisticRegression(penalty=None, max_iter=100000))]),
    'gnb': Pipeline([('select', SelectKBest(f_classif, k=k)), ('gnb', GaussianNB())]),
    'knn': Pipeline([('scale', StandardScaler()), ('select', SelectKBest(f_classif, k=k)), ('knn', KNeighborsClassifier())]),
    'svc_rbf': Pipeline([('scale', StandardScaler()), ('select', SelectKBest(f_classif, k=k)), ('svc', SVC(kernel='rbf'))]),
    'random_forest': Pipeline([('select', SelectKBest(f_classif, k=k)), ('random_forest', RandomForestClassifier(n_estimators=100, random_state=42))])
}

# Perform 5-fold cross-validation for each model
cv_results = {}
for model_name, model in classifiers.items():
    scores = cross_val_score(model, 
                             X_train[courses], 
                             y_train, 
                             cv=5,
                             scoring='accuracy',
                             verbose=0,
                             n_jobs=-1)
    cv_results[model_name] = scores
    # print(f"{model_name}: Mean -logloss = {scores.mean():.4f}, Std = {scores.std():.4f}")

# Print the cross-validation results
for model_name, scores in cv_results.items():
    print(f"{model_name}: Mean accuracy = {scores.mean():.4f}, Std = {scores.std():.4f}")

/opt/anaconda3/envs/erdos_fall_2024/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [0] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/opt/anaconda3/envs/erdos_fall_2024/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [0] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/opt/anaconda3/envs/erdos_fall_2024/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/opt/anaconda3/envs/erdos_fall_2024/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/opt/anaconda3/envs/erdos_fall_2024/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Fe

lda: Mean accuracy = 0.5849, Std = 0.0124
log_reg: Mean accuracy = 0.5846, Std = 0.0125
svc_linear: Mean accuracy = 0.5849, Std = 0.0120
qda: Mean accuracy = 0.5479, Std = 0.0102
lda_poly: Mean accuracy = 0.5879, Std = 0.0174
log_reg_poly: Mean accuracy = 0.5885, Std = 0.0180
gnb: Mean accuracy = 0.5455, Std = 0.0092
knn: Mean accuracy = 0.5398, Std = 0.0216
svc_rbf: Mean accuracy = 0.5957, Std = 0.0100
random_forest: Mean accuracy = 0.5933, Std = 0.0167


In [161]:
#Define the parameter grid
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 6, 10],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2],
    'lambda': [1, 1.5, 2],
    'alpha': [0, 0.1, 0.2],
    'n_estimators': [100],  # Changed from 'num_boost_round' to 'n_estimators'
    'objective': ['binary:logistic'],
    'eval_metric': ['logloss'],
}

# # Create the DMatrix for XGBoost
# dtrain = xgb.DMatrix(data=X_train, label=y_train)

# Initialize the XGBoost classifier
xgb_model = xgb.XGBClassifier()

# Perform randomized search
random_search = RandomizedSearchCV(estimator=xgb_model, 
                                   param_distributions=param_grid, 
                                   n_iter=500, 
                                   scoring='accuracy', 
                                   verbose=1, 
                                   n_jobs=-1)
random_search.fit(X_train, y_train)


# Print the best parameters and best score
print("Best parameters found: ", random_search.best_params_)
print("Best accuracy found: ", random_search.best_score_)

Fitting 5 folds for each of 500 candidates, totalling 2500 fits


/opt/anaconda3/envs/erdos_fall_2024/lib/python3.12/site-packages/numpy/ma/core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best parameters found:  {'subsample': 0.6, 'objective': 'binary:logistic', 'n_estimators': 100, 'min_child_weight': 1, 'max_depth': 10, 'learning_rate': 0.1, 'lambda': 1.5, 'gamma': 0, 'eval_metric': 'logloss', 'colsample_bytree': 0.6, 'alpha': 0}
Best accuracy found:  0.6656894818768068
